In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

In [2]:
# 回傳Leaderboards上某頁數的全部召喚師ID(伺服器region =0:KR, =1:NA, =2:EUW, =3:EUNE)
def get_ID(region:int, page:int)->list:
    container = []
    soup = BeautifulSoup(requests.get(f'https://{["www", "na", "euw", "eune"][region]}.op.gg/ranking/ladder/page={str(page)}').text, 'html.parser')      
    if page == 1:
        container = [i.text for i in soup.find_all("a", class_="ranking-highest__name")]
    return container + [i.text for i in soup.find_all("td", class_="select_summoner ranking-table__cell ranking-table__cell--summoner")]

In [3]:
# 回傳某召喚師ID的最近二十場內有效牌位對戰英雄及結果(左上,左野,左中,左下,左輔,右上,右野,右中,右下,右輔,0=左輸1=左贏)
def get_match(region:int, summoner_id:str)->pd.DataFrame:
    soup = BeautifulSoup(requests.get(f'https://{["www", "na", "euw", "eune"][region]}.op.gg/summoner/userName={summoner_id.replace(" ", "+")}').text, 'html.parser')
    list_match = [i.text for i in soup.find_all("div", class_="Image16")]
    list_inf = [i.get_text(strip=True) for i in soup.select('div.GameSettingInfo > div.ChampionName')] + [i.get_text(strip=True) for i in soup.find_all("div", class_="GameType")] + [i.get_text(strip=True) for i in soup.find_all("div", class_="GameResult")]
    df = pd.DataFrame(np.append(np.reshape(list_match, (-1, 10)), np.reshape(list_inf, (3, -1)).T, 1))
    df = df[~df[12].str.contains("Remake") & df[11].str.contains("Ranked")]
    df['result'] = np.where((((df[0]==df[10])|(df[1]==df[10])|(df[2]==df[10])|(df[3]==df[10])|(df[4]==df[10]))&(df[12].str.contains("Victory")))|
                            (((df[5]==df[10])|(df[6]==df[10])|(df[7]==df[10])|(df[8]==df[10])|(df[9]==df[10]))&(df[12].str.contains("Defeat"))), 1, 0)
    df.drop(df.columns[[10, 11, 12]], axis = 1, inplace = True)
    return df

In [4]:
# 回傳每個英雄腳色在OPGG的ID(查找網頁的AJAX資料時可以用到)
dict = {}
soup = BeautifulSoup(requests.get('https://www.op.gg/champion/statistics').text, 'html.parser')
for i in [link.get('href') for link in soup.select('div > a[href^="/champion/"]')]:
    r = BeautifulSoup(requests.get(f'https://www.op.gg{str(i)}').text, 'html.parser')
    a = str(r.select('h1.champion-stats-header-info__name')[0].text)
    b = str(r.find_all('script')).partition('championId: ')[2].partition('\n')[0]
    dict.setdefault(a,b)

In [ ]:
#回傳每個角色與其他角色對線的數據(有被統計出來的才會回傳)
def get_matchup():
    dict = {'亞歷斯塔': '12', '亞菲利歐': '523', '伊澤瑞爾': '81', '伊瑞莉雅': '39', '伊羅旖': '420', '伊芙琳': '28', 
            '伊莉絲': '60', '克雷德': '240', '克黎思妲': '429', '凱爾': '10', '凱特琳': '51', '凱能': '85', '凱莎': '145', 
            '剎雅': '498', '剛普朗克': '41', '加里歐': '3', '努努和威朗普': '20', '劫': '238', '勒布朗': '7', '卡力斯': '121', 
            '卡爾瑟斯': '30', '卡特蓮娜': '55', '卡瑪': '43', '卡莎碧雅': '69', '卡薩丁': '38', '卡蜜兒': '164', 
            '厄薩斯': '266', '古拉格斯': '79', '史瓦妮': '113', '吉茵珂絲': '222', '吶兒': '150', '嘉文四世': '59', 
            '圖奇': '29', '埃爾文': '427', '塔莉雅': '163', '塔里克': '44', '塔隆': '91', '墨菲特': '54', '夜曲': '56', 
            '奈德麗': '76', '奧莉安娜': '61', '好運姐': '21', '妮可': '518', '姍娜': '235', '姬亞娜': '246', '威寇茲': '161', 
            '娜米': '267', '安妮': '1', '寇格魔': '96', '崔絲塔娜': '18', '巴德': '432', '布蘭德': '63', '布郎姆': '201', 
            '布里茨': '53', '希格斯': '115', '希瓦娜': '102', '希維爾': '15', '庫奇': '42', '弗力貝爾': '106', 
            '弗拉迪米爾': '8', '悟空': '62', '悠咪': '350', '慎': '98', '慨影': '141', '拉克絲': '99', '拉姆斯': '33', 
            '提摩': '17', '斯溫': '50', '易大師': '11', '星朵拉': '134', '札克': '154', '李星': '64', '杰西': '126', 
            '枷蘿': '143', '柔依': '142', '極靈': '26', '歐拉夫': '2', '汎': '67', '沃維克': '19', '法洛士': '110', 
            '波比': '78', '泰達米爾': '23', '派克': '555', '漢默丁格': '74', '潘森': '80', '烏爾加特': '6', '烏迪爾': '77', 
            '燼': '202', '特朗德': '48', '犽宿': '157', '珍娜': '40', '瑟雷西': '412', '科加斯': '31', '約瑞科': '83', 
            '納帝魯斯': '111', '納瑟斯': '75', '索娜': '37', '索拉卡': '16', '維克特': '112', '維迦': '45', '翱銳龍獸': '136', 
            '艾克': '245', '艾妮維亞': '34', '艾希': '22', '茂凱': '57', '菲歐拉': '114', '菲艾': '254', '葛雷夫': '104', 
            '葵恩': '133', '蒙多醫生': '36', '蓋倫': '86', '薩科': '35', '藍寶': '68', '貪啃奇': '223', '費德提克': '9', 
            '賈克斯': '24', '賽勒斯': '517', '賽恩': '14', '賽特': '875', '赫克林': '120', '趙信': '5', '路西恩': '236', 
            '辛吉德': '27', '逆命': '4', '達瑞文': '119', '達瑞斯': '122', '鄂爾': '516', '銳空': '497', '鏡爪': '203', 
            '阿卡莉': '84', '阿姆姆': '32', '阿璃': '103', '阿祈爾': '268', '雷尼克頓': '58', '雷歐娜': '89', '雷玟': '92', 
            '雷珂煞': '421', '雷茲': '13', '雷葛爾': '107', '露璐': '117', '飛斯': '105', '馬爾札哈': '90', '魔甘娜': '25', 
            '魔鬥凱薩': '82', '麗珊卓': '127', '黛安娜': '131', '齊勒斯': '101', '史加納': '72'}
    tamp = []
    count = 0
    for key_a, value_a in dict.items():
        for key_b, value_b in dict.items():
            for key_c, value_c in {0:"top", 1:"jungle", 2:"mid", 3:"adc", 4:"support"}.items():
                url = f'https://www.op.gg/champion/ajax/statistics/counterChampion/championId={value_a}&targetChampionId={value_b}&position={value_c}'
                r = requests.get(url)
                soup = BeautifulSoup(r.text, 'html.parser')
                list = soup.find_all("td", class_="champion-matchup-data") 
                if list:
                    tamp.append(key_a)
                    tamp.append(key_b)
                    tamp.append(key_c)
                    if key_c == 1:
                        tamp.append(float(list[6].get_text(strip=True).replace("%", "")) * 0.01)   
                        tamp.append(None)
                        tamp.append(float(list[0].get_text(strip=True).partition(' :')[0]) / 
                                    (float(list[0].get_text(strip=True).partition(' :')[0]) + float(list[1].get_text(strip=True).partition(' :')[0])))
                        tamp.append(float(list[2].get_text(strip=True).replace("%", "")) /
                                    (float(list[2].get_text(strip=True).replace("%", "")) + float(list[3].get_text(strip=True).replace("%", ""))))
                        tamp.append(float(list[4].get_text(strip=True).replace(",", "")) /
                                    (float(list[4].get_text(strip=True).replace(",", "")) + float(list[5].get_text(strip=True).replace(",", ""))))
                    elif key_c == 4:
                        tamp.append(float(list[6].get_text(strip=True).replace("%", "")) * 0.01)         
                        tamp.append(float(list[0].get_text(strip=True).replace("%", "")) / 
                                    (float(list[0].get_text(strip=True).replace("%", "")) + float(list[1].get_text(strip=True).replace("%", ""))))
                        tamp.append(float(list[2].get_text(strip=True).partition(' :')[0]) /
                                    (float(list[2].get_text(strip=True).partition(' :')[0]) + float(list[3].get_text(strip=True).partition(' :')[0])))
                        tamp.append(float(list[4].get_text(strip=True).replace("%", "")) / 
                                    (float(list[4].get_text(strip=True).replace("%", "")) + float(list[5].get_text(strip=True).replace("%", ""))))
                        tamp.append(None)
                    else:
                        tamp.append(float(list[10].get_text(strip=True).replace("%", "")) * 0.01)         
                        tamp.append(float(list[0].get_text(strip=True).replace("%", "")) / 
                                    (float(list[0].get_text(strip=True).replace("%", "")) + float(list[1].get_text(strip=True).replace("%", ""))))
                        tamp.append(float(list[2].get_text(strip=True).partition(' :')[0]) / 
                                    (float(list[2].get_text(strip=True).partition(' :')[0]) + float(list[3].get_text(strip=True).partition(' :')[0])))
                        tamp.append(float(list[4].get_text(strip=True).replace("%", "")) / 
                                    (float(list[4].get_text(strip=True).replace("%", "")) + float(list[5].get_text(strip=True).replace("%", ""))))
                        tamp.append(float(list[6].get_text(strip=True).replace(",", "")) / 
                                    (float(list[6].get_text(strip=True).replace(",", "")) + float(list[7].get_text(strip=True).replace(",", ""))))
                count += 1
                print(u"\r任務進度: " + str((count * 100) / (148*148*5)) + "%", end="")            
    df = pd.DataFrame(np.reshape(tamp, (-1, 8)))
    df.columns = ['summoner_a', 'summoner_b', 'position', 'win_rate', 'lane_kill_rate', 'kda', 'kill_participate', 'damage']
    return df
get_matchup().to_csv("info.csv", encoding="utf-8") 

In [ ]:
# 回傳第一頁到第n頁的所有召喚師對位相關數據(我們的X)和結果(我們的Y)
def get_many_match(region:int, n:int)->pd.DataFrame:
    df = pd.DataFrame()
    count = 0
    for i in range(1, n + 1):
        for j in get_ID(region, i):
            df = df.append(get_match(region, j))
            count += 1
        print(u"\r任務一進度: " + str(count / n) + "%", end="")
    df.drop_duplicates(keep = 'first', inplace = True)
    df = df.reset_index(drop=True)
    dict = {'Aatrox': '266', 'Ahri': '103', 'Akali': '84', 'Alistar': '12', 'Amumu': '32', 'Anivia': '34', 'Annie': '1', 'Aphelios': '523',
            'Ashe': '22', 'Aurelion Sol': '136', 'Azir': '268', 'Bard': '432', 'Blitzcrank': '53', 'Brand': '63', 'Braum': '201', 'Caitlyn': '51',
            'Camille': '164', 'Cassiopeia': '69', "Cho'Gath": '31', 'Corki': '42', 'Darius': '122', 'Diana': '131', 'Dr. Mundo': '36', 'Draven': '119',
            'Ekko': '245', 'Elise': '60', 'Evelynn': '28', 'Ezreal': '81', 'Fiddlesticks': '9', 'Fiora': '114', 'Fizz': '105', 'Galio': '3', 
            'Gangplank': '41', 'Garen': '86', 'Gnar': '150', 'Gragas': '79', 'Graves': '104', 'Hecarim': '120', 'Heimerdinger': '74', 'Illaoi': '420',
            'Irelia': '39', 'Ivern': '427', 'Janna': '40', 'Jarvan IV': '59', 'Jax': '24', 'Jayce': '126', 'Jhin': '202', 'Jinx': '222',
            "Kai'Sa": '145', 'Kalista': '429', 'Karma': '43', 'Karthus': '30', 'Kassadin': '38', 'Katarina': '55', 'Kayle': '10', 'Kayn': '141',
            'Kennen': '85', "Kha'Zix": '121', 'Kindred': '203', 'Kled': '240', "Kog'Maw": '96', 'LeBlanc': '7', 'Lee Sin': '64', 'Leona': '89', 
            'Lissandra': '127', 'Lucian': '236', 'Lulu': '117', 'Lux': '99', 'Malphite': '54', 'Malzahar': '90', 'Maokai': '57', 'Master Yi': '11',
            'Miss Fortune': '21', 'Mordekaiser': '82', 'Morgana': '25', 'Nami': '267', 'Nasus': '75', 'Nautilus': '111', 'Neeko': '518',
            'Nidalee': '76', 'Nocturne': '56', 'Nunu & Willump': '20', 'Olaf': '2', 'Orianna': '61', 'Ornn': '516', 'Pantheon': '80', 'Poppy': '78',
            'Pyke': '555', 'Qiyana': '246', 'Quinn': '133', 'Rakan': '497', 'Rammus': '33', "Rek'Sai": '421', 'Renekton': '58', 'Rengar': '107',
            'Riven': '92', 'Rumble': '68', 'Ryze': '13', 'Sejuani': '113', 'Senna': '235', 'Sett': '875', 'Shaco': '35', 'Shen': '98', 
            'Shyvana': '102', 'Singed': '27', 'Sion': '14', 'Sivir': '15', 'Skarner':'72', 'Sona': '37', 'Soraka': '16', 'Swain': '50', 'Sylas': '517', 
            'Syndra': '134', 'Tahm Kench': '223', 'Taliyah': '163', 'Talon': '91', 'Taric': '44', 'Teemo': '17', 'Thresh': '412', 'Tristana': '18',
            'Trundle': '48', 'Tryndamere': '23', 'Twisted Fate': '4', 'Twitch': '29', 'Udyr': '77', 'Urgot': '6', 'Varus': '110', 'Vayne': '67',
            'Veigar': '45', "Vel'Koz": '161', 'Vi': '254', 'Viktor': '112', 'Vladimir': '8', 'Volibear': '106', 'Warwick': '19', 'Wukong': '62', 
            'Xayah': '498', 'Xerath': '101', 'Xin Zhao': '5', 'Yasuo': '157', 'Yorick': '83', 'Yuumi': '350', 'Zac': '154', 'Zed': '238',
            'Ziggs': '115', 'Zilean': '26', 'Zoe': '142', 'Zyra': '143'}
    container = []
    count = 0
    for index, row in df.iterrows():
        tamp = []
        for i in range(0, 5):
            url = f'https://www.op.gg/champion/ajax/statistics/counterChampion/championId={dict[row[i]]}&targetChampionId={dict[row[i + 5]]}&position={["top", "jungle", "mid", "adc", "support"][i]}'
            list = BeautifulSoup(requests.get(url).text, 'html.parser').find_all("td", class_="champion-matchup-data")
            if list:
                if i == 1:
                    tamp.append(float(list[6].get_text(strip=True).replace("%", "")) * 0.01)         
                    tamp.append(float(list[0].get_text(strip=True).partition(' :')[0]) / 
                                     (float(list[0].get_text(strip=True).partition(' :')[0]) + float(list[1].get_text(strip=True).partition(' :')[0])))
                    tamp.append(float(list[2].get_text(strip=True).replace("%", "")) /
                                     (float(list[2].get_text(strip=True).replace("%", "")) + float(list[3].get_text(strip=True).replace("%", ""))))
                    tamp.append(float(list[4].get_text(strip=True).replace(",", "")) /
                                     (float(list[4].get_text(strip=True).replace(",", "")) + float(list[5].get_text(strip=True).replace(",", ""))))
                elif i == 4:
                    tamp.append(float(list[6].get_text(strip=True).replace("%", "")) * 0.01)         
                    tamp.append(float(list[0].get_text(strip=True).replace("%", "")) / 
                                     (float(list[0].get_text(strip=True).replace("%", "")) + float(list[1].get_text(strip=True).replace("%", ""))))
                    tamp.append(float(list[2].get_text(strip=True).partition(' :')[0]) /
                                     (float(list[2].get_text(strip=True).partition(' :')[0]) + float(list[3].get_text(strip=True).partition(' :')[0])))
                    tamp.append(float(list[4].get_text(strip=True).replace("%", "")) / 
                                     (float(list[4].get_text(strip=True).replace("%", "")) + float(list[5].get_text(strip=True).replace("%", ""))))
                else:
                    tamp.append(float(list[10].get_text(strip=True).replace("%", "")) * 0.01)         
                    tamp.append(float(list[0].get_text(strip=True).replace("%", "")) / 
                                     (float(list[0].get_text(strip=True).replace("%", "")) + float(list[1].get_text(strip=True).replace("%", ""))))
                    tamp.append(float(list[2].get_text(strip=True).partition(' :')[0]) / 
                                     (float(list[2].get_text(strip=True).partition(' :')[0]) + float(list[3].get_text(strip=True).partition(' :')[0])))
                    tamp.append(float(list[4].get_text(strip=True).replace("%", "")) / 
                                     (float(list[4].get_text(strip=True).replace("%", "")) + float(list[5].get_text(strip=True).replace("%", ""))))
                    tamp.append(float(list[6].get_text(strip=True).replace(",", "")) / 
                                     (float(list[6].get_text(strip=True).replace(",", "")) + float(list[7].get_text(strip=True).replace(",", ""))))
            else:
                break
            if i == 4:
                container += tamp
                container.append(row["result"])
        count += 1
        print(u"\r任務二進度: " + str(count * 100 / df.shape[0]) + "%", end="")
    df = pd.DataFrame(np.reshape(container, (-1, 24)))
    df = df.reset_index(drop=True)
    df.columns = ['top_win_rate', 'top_lane_kill_rate', 'top_kda', 'top_kill_participate', 'top_damage',
                  'jg_win_rate', 'jg_kda', 'jg_kill_participate', 'jg_damage', 
                  'mid_win_rate', 'mid_lane_kill_rate', 'mid_kda', 'mid_kill_participate', 'mid_damage', 
                  'adc_win_rate', 'adc_lane_kill_rate', 'adc_kda', 'adc_kill_participate', 'adc_damage',
                  'sup_win_rate', 'sup_lane_kill_rate', 'sup_kda', 'sup_kill_participate', 'result']
    return df
get_many_match(2, 25).to_csv("euw.csv")